# Introduction

> The script automates gathering of tweet data from the archives

## Packages

In [1]:
#import packages 
import pandas as pd 
import numpy as np
import os 
import time
import requests
import itertools    
import json
import re

from datetime import date
from datetime import datetime
from datetime import timedelta

#import data scraping packages 
import tweepy
# from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import Cursor
from tweepy import API

#import web scrapping packages 
import bs4 as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from selenium_stealth import stealth
import undetected_chromedriver as uc



#sns module 
import snscrape.modules.twitter as sntwitter
import snscrape.modules.facebook as snfacebook
import snscrape.modules.instagram as sninstagram

#postgresql database packages 
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine

import csv
from io import StringIO

#check the start time
start_time = datetime.now()

## Extract Time

In [2]:
#generate the 
# expected_days =365*1
number_of_dates = 365*2
end_date = date.today() #-timedelta(days=expected_days)
start_date =  end_date - timedelta(days=number_of_dates)
duration = end_date - start_date

print()
print()
print(f"The start date: {start_date} \n end date is: {end_date} and \n time duration is: {duration}")



The start date: 2020-09-07 
 end date is: 2022-09-07 and 
 time duration is: 730 days, 0:00:00


In [3]:
#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%B %d %Y %H %M %S")

#print date 
print()
print()
print("Todays date: ", formated_date)



Todays date:  September 07 2022 13 13 41


### Keywords

> The following are search keywords for cyber security concerns 

In [4]:
#define variables 
keywords_list = ["Trojan", "Spyware", "ransomware", "Botnet", "Defense", "Evasion","ingress tool transfer", "Malicious link","firmware", "Bruteforce", "automated exfiltration", 
                 "DNSSEC traffic", "channel attacks", "DHCP spoofing","Protocol impersonation", "Denial of Service", "Phishing", "Spearphishing", "OS Credential Dumping"]

#search words 
#fornat the sland 
print()
print()
print(f"{'*'*50} Keyword list {'*'*60}")
print(keywords_list)

print()
print()
#search words 
print(f"{'*'*50} Joined Keyword List {'*'*50}")
search_word = "|".join([j.lower() for j in keywords_list])
print(search_word)





************************************************** Keyword list ************************************************************
['Trojan', 'Spyware', 'ransomware', 'Botnet', 'Defense', 'Evasion', 'ingress tool transfer', 'Malicious link', 'firmware', 'Bruteforce', 'automated exfiltration', 'DNSSEC traffic', 'channel attacks', 'DHCP spoofing', 'Protocol impersonation', 'Denial of Service', 'Phishing', 'Spearphishing', 'OS Credential Dumping']


************************************************** Joined Keyword List **************************************************
trojan|spyware|ransomware|botnet|defense|evasion|ingress tool transfer|malicious link|firmware|bruteforce|automated exfiltration|dnssec traffic|channel attacks|dhcp spoofing|protocol impersonation|denial of service|phishing|spearphishing|os credential dumping


### African Countries

In [5]:
#countries involved
name = ['Algeria','Angola','Benin','Botswana','Burkina_Faso','Burundi','Cameroon','Cape_Verde',
             'Central_African_Republic','Chad','Comoros','Democratic_Republic_of_the_Congo','Djibouti','Egypt',
             'Equatorial_Guinea',' Eritrea',' Ethiopia',' Gabon',' Gambia',' Ghana','Guinea',' Guinea-Bissau',
             'Ivory_Coast','Kenya',' Lesotho',' Liberia',' Libya',' Madagascar',' Malawi',' Mali',' Mauritania','Mauritius',
             'Morocco',' Mozambique',' Namibia',' Niger',' Nigeria','Republic of the Congo',' Rwanda','Sao_Tome_and_Principe',
             'Senegal',' Seychelles',' Sierra_Leone',' Somalia',' South_Africa',' Sudan','South_Sudan','Swaziland',' Tanzania',' Togo',
             'Tunisia',' Uganda',' Zambia',' Zimbabwe']

# #strip any white spaces 
country_name = [x.strip().lower().replace('_','-').replace(' ','-') for x in name]

#search words 
countries = "|".join([j.strip().lower() for j in name])

In [6]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [7]:
 driver = webdriver.Chrome(r'C:\Users\chromedriver_win32\chromedriver.exe')

C:\Users\moses\AppData\Local\Temp\ipykernel_15716\1516475625.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\chromedriver_win32\chromedriver.exe')


## Place Specific 

> Countries towns and cities co-ordinates [link](https://www.countrycoordinate.com/)

In [8]:
#get the data in a dataframe 
coordinates_df =  pd.DataFrame()
count = 0

for country in country_name:
    #get the url
    url = f"https://www.countrycoordinate.com/country-{country}/"
    count +=1
    print(f"Num: {count}; link: {url}") 
    try:
        driver = uc.Chrome(use_subprocess=True)
        driver.get(url)
        time.sleep(5)
        soup = bs.BeautifulSoup(driver.page_source,'lxml')

        # get data table & save as dataframe
        table_val = soup.find("table",{'table table-bordered table-hover'})
        table_val_df = pd.read_html(str(table_val))[0]

        table_val_df.columns = ['city', 'coords']
        table_val_df['country'] = country
        table_val_df['url'] = url
        table_val_df[['lat', 'lon']] = table_val_df['coords'].str.split('/', expand = True)
        table_val_df.drop(columns = ['coords'], inplace = True)
        if table_val_df.shape[0]>1:
            coordinates_df = pd.concat([coordinates_df,table_val_df])
        else:
            pass


        #get the number of other pages 
        page_num_table = soup.find_all("table", {"class":"table table-bordered centered"})



          #loop through other pages
        print(f"{'*'*15} base page done {'*'*15}")
        print(f"{'*'*15} Other pages {'*'*15}")
        for tab in page_num_table:
            row = tab.find_all('a')
            row = [i.text.strip() for i in row]
            row = row[:-1]
            print(f"{'*'*15} pages to extract {len(row)+1} {'*'*15}")

            for page in row: 
                next_page_url = url+f"?page={page}"
                print(next_page_url)

                #get the data 
                driver.get(next_page_url)
                time.sleep(2)
                soup_page = bs.BeautifulSoup(driver.page_source, 'lxml')

                # get data table & save as dataframe
                table_val_next = soup_page.find("table",{'table table-bordered table-hover'})
                table_val_df_1 = pd.read_html(str(table_val_next))[0]
                table_val_df_1.columns = ['city', 'coords']
                table_val_df_1['country'] = country
                table_val_df_1['url'] = url
                table_val_df_1[['lat', 'lon']] = table_val_df_1['coords'].str.split('/', expand = True)
                table_val_df_1.drop(columns = ['coords'], inplace = True)
                if table_val_df_1.shape[0]>1:
                    coordinates_df = pd.concat([coordinates_df,table_val_df_1])
                else:
                    pass
                
    except Exception as e:
        print(f'The error is: {str(e)}')
        pass
    
#clean the data 

#reset the index
coordinates_df.reset_index(drop = True, inplace = True)

#clean the data 
coordinates_df['city'] = coordinates_df['city'].str.split('(', expand = True)[0]

#drop duplicate towns
print(f"Rows before droping duplicates {coordinates_df.shape[0]}")
coordinates_df.drop_duplicates(subset = ['city', 'country'], inplace = True, keep = 'last')
print(f"Rows befor droping duplicates {coordinates_df.shape[0]}")

#print the sample 
coordinates_df.sample(3)

Num: 1; link: https://www.countrycoordinate.com/country-algeria/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 7 ***************
https://www.countrycoordinate.com/country-algeria/?page=2
https://www.countrycoordinate.com/country-algeria/?page=3
https://www.countrycoordinate.com/country-algeria/?page=4
https://www.countrycoordinate.com/country-algeria/?page=5
https://www.countrycoordinate.com/country-algeria/?page=12
https://www.countrycoordinate.com/country-algeria/?page=13
Num: 2; link: https://www.countrycoordinate.com/country-angola/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 2 ***************
https://www.countrycoordinate.com/country-angola/?page=2
Num: 3; link: https://www.countrycoordinate.com/country-benin/
*************** base page done ***************
*************** Other pages ***************
*************** pages to e

*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 3 ***************
https://www.countrycoordinate.com/country-libya/?page=2
https://www.countrycoordinate.com/country-libya/?page=3
Num: 28; link: https://www.countrycoordinate.com/country-madagascar/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 6 ***************
https://www.countrycoordinate.com/country-madagascar/?page=2
https://www.countrycoordinate.com/country-madagascar/?page=3
https://www.countrycoordinate.com/country-madagascar/?page=4
https://www.countrycoordinate.com/country-madagascar/?page=5
https://www.countrycoordinate.com/country-madagascar/?page=6
Num: 29; link: https://www.countrycoordinate.com/country-malawi/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 2 ***************
https://www.countrycoor

Num: 52; link: https://www.countrycoordinate.com/country-uganda/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 4 ***************
https://www.countrycoordinate.com/country-uganda/?page=2
https://www.countrycoordinate.com/country-uganda/?page=3
https://www.countrycoordinate.com/country-uganda/?page=4
Num: 53; link: https://www.countrycoordinate.com/country-zambia/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 4 ***************
https://www.countrycoordinate.com/country-zambia/?page=2
https://www.countrycoordinate.com/country-zambia/?page=3
https://www.countrycoordinate.com/country-zambia/?page=4
Num: 54; link: https://www.countrycoordinate.com/country-zimbabwe/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 3 ***************
https://www.countrycoordinate.com/c

,city,country,url,lat,lon
1915,Takamaka,seychelles,https://www.countrycoordinate.com/country-seyc...,-4.784369,55.508101
714,Makadi Bay,egypt,https://www.countrycoordinate.com/country-egypt/,26.984847,33.902051
481,Boali,central-african-republic,https://www.countrycoordinate.com/country-cent...,4.888774,18.142867


In [9]:
country_df = coordinates_df[coordinates_df['country'] == 'kenya']
# country_df = coordinates_df.copy()

> Extract tweets with keywords

> try conditional loop by intializing 

In [14]:
#create and empty df
df_x = pd.DataFrame()

#define the max keywords
maxcounts = 100
count = 0

try: 
    for row in country_df.index[:1]:
            country = country_df['country'][row]
            nearby_location = country_df['city'][row]
            lat = country_df['lat'][row]
            lon = country_df['lon'][row]
            circumfrence = "20km"
            coor = lat+","+lon+","+circumfrence
            coor = format(coor)
            print()
            print(f"country: {country}, nearby_loc: {nearby_location},  coordinates: {lat+', '+lon}, radius: {circumfrence}")
            
            try:
                for word in keywords_list:
                    count +=1
                    
                    print(f"{'*'*20} instance within: {count};  word: {word} {'*'*20}")
                    scraped_tweets = sntwitter.TwitterSearchScraper(f'{str(word)} since:{start_date} until:{end_date} geocode:"{coor}"').get_items()

                    sliced_scraped_tweets = itertools.islice(scraped_tweets, maxcounts)  
                    df_coord = pd.DataFrame(sliced_scraped_tweets)
                    df_coord.columns = df_coord.columns.str.lower()
                    df_coord['keyword'] = word
                    df_coord['nearby_location'] = nearby_location
                    df_coord['country'] = country
                    df_x = pd.concat([df_x, df_coord])
                    
            except Exception as e:
                print(f'The error was: {str(e)}')
except Exception as e:
    print(f'The error was: {str(e)}')


country: kenya, nearby_loc: Nairobi,  coordinates: -1.292066 ,  36.821946, radius: 20km
******************** instance within: 1;  word: Trojan ********************
******************** instance within: 2;  word: Spyware ********************
******************** instance within: 3;  word: ransomware ********************
******************** instance within: 4;  word: Botnet ********************
******************** instance within: 5;  word: Defense ********************
******************** instance within: 6;  word: Evasion ********************
******************** instance within: 7;  word: ingress tool transfer ********************
******************** instance within: 8;  word: Malicious link ********************
******************** instance within: 9;  word: firmware ********************
******************** instance within: 10;  word: Bruteforce ********************
******************** instance within: 11;  word: automated exfiltration ********************
******************** 

In [13]:
df_x.tail(5)

,url,date,content,renderedcontent,id,user,replycount,retweetcount,likecount,quotecount,...,inreplytotweetid,inreplytouser,mentionedusers,coordinates,place,hashtags,cashtags,keyword,nearby_location,country
37,https://twitter.com/WeAreComputech/status/1346...,2021-01-06 13:21:52+00:00,"New Year, New Cyber Security Goals. Don't be l...","New Year, New Cyber Security Goals. Don't be l...",1.346809e+18,"{'username': 'WeAreComputech', 'id': 738260726...",0.0,0.0,0.0,0.0,...,NaN,None,None,"{'longitude': 36.6645733, 'latitude': -1.3890532}","{'fullName': 'Nairobi, Kenya', 'name': 'Nairob...","[wednesdaythought, Assange]",None,Phishing,Nairobi,kenya
38,https://twitter.com/WeAreComputech/status/1337...,2020-12-11 13:54:47+00:00,We’ve reported a whopping 73% growth in Email ...,We’ve reported a whopping 73% growth in Email ...,1.337395e+18,"{'username': 'WeAreComputech', 'id': 738260726...",0.0,0.0,0.0,0.0,...,NaN,None,None,"{'longitude': 36.79512286200224, 'latitude': -...","{'fullName': 'Computech HQ Nairobi', 'name': '...","[COVID19, wearecomputech, ciscopartner, ciscos...",None,Phishing,Nairobi,kenya
39,https://twitter.com/Feloski_/status/1321157479...,2020-10-27 18:31:03+00:00,@Safaricom_Care \nSomeone is running extensive...,@Safaricom_Care \nSomeone is running extensive...,1.321157e+18,"{'username': 'Feloski_', 'id': 44633225, 'disp...",2.0,0.0,0.0,0.0,...,NaN,None,"[{'username': 'Safaricom_Care', 'id': 32066218...","{'longitude': 36.6645733, 'latitude': -1.3890532}","{'fullName': 'Nairobi, Kenya', 'name': 'Nairob...","[SafaricomSpin, Safanniversaryprizes]",None,Phishing,Nairobi,kenya
40,https://twitter.com/MaudhuiHouse/status/131168...,2020-10-01 15:12:57+00:00,Customers must also be informed of their role ...,Customers must also be informed of their role ...,1.311686e+18,"{'username': 'MaudhuiHouse', 'id': 94488234361...",1.0,3.0,2.0,0.0,...,1311685403146153984.0,"{'username': 'MaudhuiHouse', 'id': 94488234361...",None,"{'longitude': 36.6645733, 'latitude': -1.3890532}","{'fullName': 'Nairobi, Kenya', 'name': 'Nairob...",None,None,Phishing,Nairobi,kenya
41,https://twitter.com/iKaizoku_/status/130361570...,2020-09-09 08:46:19+00:00,@Lord_Olelieza 😂😂😂😂 Those are skids and social...,@Lord_Olelieza 😂😂😂😂 Those are skids and social...,1.303616e+18,"{'username': 'iKaizoku_', 'id': 326619625, 'di...",2.0,0.0,0.0,0.0,...,1303614874795356160.0,"{'username': 'Lord_Olelieza', 'id': 413038728,...","[{'username': 'Lord_Olelieza', 'id': 413038728...","{'longitude': 36.6645733, 'latitude': -1.3890532}","{'fullName': 'Nairobi, Kenya', 'name': 'Nairob...",None,None,Phishing,Nairobi,kenya


> Clean the extracted tweets

In [15]:

#subset the data
df = df_x[['id', 'date', 'content', 'renderedcontent', 'user', 'replycount','retweetcount', 'likecount', 'quotecount',
           'url', 'lang', 'retweetedtweet', 'quotedtweet','mentionedusers', 'coordinates', 'place', 'hashtags', 'keyword', 
           'nearby_location', 'country']]

#format the date
df['date'] = df['date'].apply(lambda x: x.strftime("%d-%b-%Y %H:%M:%S"))

# getting user's location
df.reset_index(drop = True, inplace = True)
df['username'] = df['user'].apply(lambda x: x['username'])
df['displayname'] = df['user'].apply(lambda x: x['displayname'])
df['user_id'] = df['user'].apply(lambda x: x['id'])
df['user_location'] = df['user'].apply(lambda x: x['location'])
df['description'] = df['user'].apply(lambda x: x['description'])
df['followersCount'] = df['user'].apply(lambda x: x['followersCount'])
df['friendsCount'] = df['user'].apply(lambda x: x['friendsCount'])
df['statusesCount'] = df['user'].apply(lambda x: x['statusesCount'])
df['favouritesCount'] = df['user'].apply(lambda x: x['favouritesCount'])
df['protected'] = df['user'].apply(lambda x: x['protected'])

#mentions 
pattern_1 = r'#(\w+)'
pattern_2 = r'@(\w+)'
# df['hashtag'] = df['content'].apply(lambda x: re.findall(pattern_1, x))
df['mentions'] = df['content'].apply(lambda x: re.findall(pattern_2, x))

#drop some columns 
df.drop(columns = ['user', 'mentionedusers'], inplace = True)


#clean the data 
##lower the text
df['content'] = df['content'].apply(lambda x: x.lower())
df['content'] = df['content'].apply(lambda x: x.strip())
df['content'] = df['content'].apply(lambda x: x.replace("  ", " "))
df['content'] = df['content'].apply(lambda x: x.replace("\n", ""))

##strip white spaces 
df.columns = df.columns.str.strip()

#lower all the names 
df.colums = df.columns.str.lower()


#filter words that contained in the list 
df['search_word'] = np.where(df['content'].str.contains(search_word),'Yes','No')

##drop duplicate tweets if any 
print(f'The number of tweets before droping duplicates are: {df.shape[0]}')
df = df.drop_duplicates(subset=['date', 'id', 'content', 'username', 'user_location', 'renderedcontent', 'keyword', 'lang', 'url', 'user_id'], keep= 'last')
print(f'The number of tweets after droping duplicates are: {df.shape[0]}')

#print the data
df.sample(5)

C:\Users\moses\AppData\Local\Temp\ipykernel_15716\4275255497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].apply(lambda x: x.strftime("%d-%b-%Y %H:%M:%S"))
C:\Users\moses\AppData\Local\Temp\ipykernel_15716\4275255497.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username'] = df['user'].apply(lambda x: x['username'])
C:\Users\moses\AppData\Local\Temp\ipykernel_15716\4275255497.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

The number of tweets before droping duplicates are: 314
The number of tweets after droping duplicates are: 314


,id,date,content,renderedcontent,replycount,retweetcount,likecount,quotecount,url,lang,...,user_id,user_location,description,followersCount,friendsCount,statusesCount,favouritesCount,protected,mentions,search_word
173,1.566675e+18,05-Sep-2022 06:28:22,@_thefigen yep thats a defense lawyer..,@_TheFigen yep thats a defense lawyer..,0.0,0.0,0.0,0.0,https://twitter.com/that_bloke_kiri/status/156...,en,...,1271015437,"Nairobi,kenya",#kenyan #skeptic #lovefacts #lovebeer #loveuni...,1161,4388,33371,43071,False,[_TheFigen],Yes
39,1.418627e+18,23-Jul-2021 17:39:32,@cjmarthakoome is @kenyajudiciary trojan horse...,@CJMarthaKoome is @Kenyajudiciary trojan horse...,0.0,0.0,0.0,0.0,https://twitter.com/frankmengi/status/14186268...,en,...,38147722,"Nairobi, Kenya",Commentary ◇ Politics ♧ Technology ♤ Bar Talk ...,445,2765,9836,1095,False,"[CJMarthaKoome, Kenyajudiciary]",Yes
207,1.539597e+18,22-Jun-2022 13:12:55,@abuoralenn @lemayianleiyan you remember how c...,@abuoralenn @LemayianLeiyan You remember how C...,1.0,0.0,21.0,0.0,https://twitter.com/okakafelix/status/15395972...,en,...,57274620,"Nairobi, Kenya",Photographer\n\n\nBookings info@okakafelix.com,1549,2438,14094,1534,False,"[abuoralenn, LemayianLeiyan]",Yes
260,1.387524e+18,28-Apr-2021 21:46:57,new firmware version is 3e751,New Firmware version is 3E751,0.0,1.0,0.0,0.0,https://twitter.com/MacStickerPlug/status/1387...,en,...,1159147095602937857,127.0.0.1,Technophile @infosecfixer| PC Architect For Ga...,2373,997,9324,6552,False,[],Yes
272,1.567175e+18,06-Sep-2022 15:37:52,@_cerullomomanyi cat phishing ndo umeeka mbele,@_Cerullomomanyi Cat phishing ndo umeeka mbele,1.0,0.0,0.0,0.0,https://twitter.com/Daaz_Baba/status/156717525...,en,...,282453445,"Nairobi,Kenya",🇰🇪▪️GrandsonOfaFox▪️RugbyPundit @LFC @EASPORTS...,1305,1508,26223,25116,False,[_Cerullomomanyi],Yes


In [16]:
xy = df[df['search_word'] == 'Yes']
xy['keyword'].value_counts()

Defense              100
Evasion               72
Trojan                48
Phishing              42
ransomware            18
firmware              15
Spyware                7
Denial of Service      4
Malicious link         1
Bruteforce             1
Name: keyword, dtype: int64

In [19]:
for tweet in df.index:
    if df['search_word'][tweet] == 'Yes':
        print(f"keyword: {df['keyword'][tweet]} \n Extracted date: {df['date'][tweet]} \n tweet:> {df['renderedcontent'][tweet]} \n link {df['url'][tweet]}")
        print(' '*120)
        print('-'*120)

keyword: Trojan 
 Extracted date: 06-Sep-2022 18:15:09 
 tweet:> @trojan_ragdde @Kenyans Wewe kacheze huko bana. 
 link https://twitter.com/Lemeiyan/status/1567214837343698946
                                                                                                                        
------------------------------------------------------------------------------------------------------------------------
keyword: Trojan 
 Extracted date: 06-Sep-2022 08:57:02 
 tweet:> @MissTichie @MarthaKarua Martha was a Trojan Horse sent to fuck up Azimio😏I like the game that was played here🤣 
 link https://twitter.com/TToryquer/status/1567074381938974725
                                                                                                                        
------------------------------------------------------------------------------------------------------------------------
keyword: Trojan 
 Extracted date: 05-Sep-2022 16:54:37 
 tweet:> @citizentvkenya Trojan horse 
 lin

> Tweets containing keywords

In [20]:
df['search_word'].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'

Yes    98.1%
No      1.9%
Name: search_word, dtype: object

> Drop the tweets that do not have the keywords

In [30]:
#filter out the only data with matching keyword
data = df[df['search_word']!='No']
data.reset_index(drop = True, inplace = True)
data.head(5)

,id,date,content,renderedcontent,replycount,retweetcount,likecount,quotecount,url,lang,...,user_id,user_location,description,followersCount,friendsCount,statusesCount,favouritesCount,protected,mentions,search_word
0,1.420751e+18,29-Jul-2021 14:19:04,la società per azioni proprietaria della socie...,La società per azioni proprietaria della socie...,0.0,0.0,0.0,0.0,https://twitter.com/abdennour68/status/1420750...,it,...,2174776405,Italie,Che mondo sarebbe senza COVID19?,140,668,1588,187,False,[],Yes
1,1.394687e+18,18-May-2021 16:10:30,@billieeilish netanyahu threatens to hit the p...,@billieeilish Netanyahu threatens to hit the P...,1.0,0.0,1.0,0.0,https://twitter.com/DMDM90764296/status/139468...,en,...,1267934603282612231,,Love your self,1,27,56,178,False,[billieeilish],Yes
2,1.394686e+18,18-May-2021 16:08:38,@bbcworld netanyahu threatens to hit the pales...,@BBCWorld Netanyahu threatens to hit the Pales...,0.0,0.0,0.0,0.0,https://twitter.com/DMDM90764296/status/139468...,en,...,1267934603282612231,,Love your self,1,27,56,178,False,[BBCWorld],Yes
3,1.394686e+18,18-May-2021 16:05:43,@alarabiya_shows @taherbaraka netanyahu threat...,@AlArabiya_shows @taherbaraka Netanyahu threat...,0.0,0.0,0.0,0.0,https://twitter.com/DMDM90764296/status/139468...,en,...,1267934603282612231,,Love your self,1,27,56,178,False,"[AlArabiya_shows, taherbaraka]",Yes
4,1.394685e+18,18-May-2021 16:04:26,@mtv @avrillavigne netanyahu threatens to hit ...,@MTV @AvrilLavigne Netanyahu threatens to hit ...,0.0,0.0,0.0,0.0,https://twitter.com/DMDM90764296/status/139468...,en,...,1267934603282612231,,Love your self,1,27,56,178,False,"[MTV, AvrilLavigne]",Yes


In [31]:
for tweet in data.index:
#     if df['search_word'] != 'Yes':
    print(f"tweet:> {data['renderedcontent'][tweet]} \n link {data['url'][tweet]}")
    print(' '*120)
    print('-'*120)
    

tweet:> La società per azioni proprietaria della società israeliana di spyware Pegasus NSO sarà liquidata english.alaraby.co.uk/news/firm-owni… 
 link https://twitter.com/abdennour68/status/1420750729124278282
                                                                                                                        
------------------------------------------------------------------------------------------------------------------------
tweet:> @billieeilish Netanyahu threatens to hit the Palestinians with 12,000 rockets this evening
Israel is killing childrens with no marcy and America is supporting theme by saying it's self-defense.
#GhazaUnderAttach 
#PalestineUnderAttack 
#FreePalestine 
#IsraeliCrimes 
#isrealterrorist https://t.co/0v1hr0AYaB 
 link https://twitter.com/DMDM90764296/status/1394686845758451712
                                                                                                                        
-------------------------------------------

## Save the File

> Create file Name

In [35]:
#generate a filename 
name = f"african_command_center_tweets"

#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%d_%b").lower()

#extension
extension = ".xlsx"

#generate the name
new_filename = name+"_"+formated_date
old_filename = name+"_"+formated_date+"_"+"oldfile"
print(old_filename ,"\n", new_filename)

african_command_center_tweets_26_aug_oldfile 
 african_command_center_tweets_26_aug


> Write data to Postgresql

In [18]:
#create the connection 

db_host = 'research-dev-db.cluster-c30rc84jwwhk.us-east-2.rds.amazonaws.com'
# db_host = '192.168.8.221'
db_port = 5432
db_password = 'researchpassword'
db = 'postgres'
db_user = 'postgres'


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}')

#write the data if the table exists , replace the data
try:
    data.to_sql(new_filename, engine, method=psql_insert_copy, if_exists='replace')
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

> write the african countries cities

In [19]:
#write the data if the table exists , replace the data
table_name = "african_countries_towns_and_cities"

#write the data into db
try:
    coordinates_df.to_sql(table_name, engine, method=psql_insert_copy, if_exists='replace')
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

> Rename file if present in the directory 

In [20]:
#rename the file if is in directory 
try:
  os.rename(old_filename+'.xlsx', new_filename+'xlsx')
except Exception as e:
  print(f"The error is: {str(e)}")
  pass

The error is: [WinError 2] The system cannot find the file specified: 'african_command_center_tweets_26_aug_oldfile.xlsx' -> 'african_command_center_tweets_26_augxlsx'


> Save the File

In [21]:
#write the file
try:
    data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
except BaseException as e:
    print('-'*20)
    print(f"The error is {str(e)}")
    pass

In [22]:
# #write the file
# try:
# #     data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
# except BaseException as e:
#     print('-'*20)
#     print(f"The error is {str(e)}")
#     pass

## Time Spend

In [23]:
#get the time used to proccess the data
end_time = datetime.now()
duration = end_time - start_time
print(f"The stopped at: {end_time} '\n' Started at: {end_time} '\n' and took {duration} to run")

The stopped at: 2022-08-26 04:34:45.556083 '
' Started at: 2022-08-26 04:34:45.556083 '
' and took 2 days, 7:47:54.263342 to run
